## **Proyecto Adonde Vivir**

In [3]:
## Librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from tqdm import tqdm  # pip install tqdm
import datetime as dt

## Request
import requests
from ubigeos_peru import Ubigeo as ubg

In [2]:

## Inputs. 

# Preguntar portal
while True:
    portal_numero = input("Seleccione portal (1=A Donde Vivir, 2=Urbania): ")

    if portal_numero == "1":
        portal = "adondevivir"
        print("Se usará datos de A Donde Vivir")
        break
    elif portal_numero == "2":
        portal = "urbania"
        print("Se usará datos de Urbania")
        break
    else:
        print("✖ Opción no válida. Intente nuevamente.")

# Preguntar tipo de operación
while True:
    operacion_num = input("Seleccione operación (1=Venta, 2=Alquiler): ")

    if operacion_num == "1":
        operacion = "venta"
        print("Se consultará propiedades en Venta")
        break
    elif operacion_num == "2":
        operacion = "alquiler"
        print("Se consultará propiedades en Alquiler")
        break
    else:
        print("✖ Opción no válida. Intente nuevamente.")

# Preguntar tipo de inmueble
while True:
    inmueble_num = input("Seleccione el tipo de inmueble (1=Departamento, 2=Casa, 3=Terrenos, 4=Local Comercial): ")

    if inmueble_num == "1":
        inmueble = "departamentos"
        print("Se consultará Departamentos")
        break
    elif inmueble_num == "2":
        inmueble = "casas"
        print("Se consultará Casas")
        break
    elif inmueble_num == "3":
        inmueble = "terrenos"
        print("Se consultará Terrenos")
        break
    elif inmueble_num == "4":
        inmueble = "locales-comerciales"
        print("Se consultará Locales Comerciales")
        break
    else:
        print("✖ Opción no válida. Intente nuevamente.")


hoy = dt.date.today()

# Resultado final
print(f"\n👉 Portal seleccionado: {portal}, Operación: {operacion} y Inmueble: {inmueble}, fecha solicitud: {hoy}")


Se usará datos de Urbania
Se consultará propiedades en Alquiler
Se consultará Departamentos

👉 Portal seleccionado: urbania, Operación: alquiler y Inmueble: departamentos, fecha solicitud: 2025-11-24


### **Enlaces Distritos**

In [4]:
distritos = [
    ##"Miraflores" , "Ate Vitarte" 
    # ## distritos lima.
    "Ancon", "Ate Vitarte"
    # , "Barranco", "Brena", "Carabayllo", "Chaclacayo", "Chorrillos",
    # "Cieneguilla", "Comas", "El Agustino", "Independencia", "Jesus Maria",
    # "La Molina", "La Victoria", "Lima Cercado", "Lince", "Los Olivos", "Chosica Lurigancho", "Lurin",
    # "Magdalena", "Miraflores", "Pachacamac", "Pucusana", "Pueblo Libre",
    # "Puente Piedra", "Punta Hermosa", "Punta Negra", "Rimac", "San Bartolo",
    # "San Borja", "San Isidro", "San Juan de Lurigancho", "San Juan de Miraflores",
    # "San Luis", "San Martin de Porres", "San Miguel", "Santa Anita", "Santa Maria del Mar", "Santa Rosa",
    # "Santiago de Surco", "Surquillo", "Villa El Salvador", "Villa Maria del Triunfo"
    ## distritos callao
    # "Callao", "Bellavista", "Carmen de la Legua Reynoso", "La Perla", "La Punta", 
    # "Ventanilla", "Mi Peru"
]

In [5]:
## Funcion Distrito Procesado
def distrito_procesado(distrito):
    distrito = distrito.lower().replace(" ", "-")
    return distrito

web_distrito = []

if portal == "adondevivir":
    
    for distrito in distritos:
        web = f"https://www.adondevivir.com/{inmueble}-en-{operacion}-en-{distrito_procesado(distrito)}.html" ## a donde vivir. 
        web_distrito.append({
        "distrito": distrito, 
        "web": web,
        "portal": portal,
        "operacion":operacion, 
        "inmueble": inmueble
        })
    
else:
    for distrito in distritos:
        web =  f"https://urbania.pe/buscar/{operacion}-de-{inmueble}-en-{distrito_procesado(distrito)}--lima--lima" ## urbania ok
        web_distrito.append({
        "distrito": distrito, 
        "web": web,
        "portal": portal,
        "operacion":operacion, 
        "inmueble": inmueble
        })
        
## Resultados:
web_distrito[:5] 

[{'distrito': 'Ancon',
  'web': 'https://urbania.pe/buscar/alquiler-de-departamentos-en-ancon--lima--lima',
  'portal': 'urbania',
  'operacion': 'alquiler',
  'inmueble': 'departamentos'},
 {'distrito': 'Ate Vitarte',
  'web': 'https://urbania.pe/buscar/alquiler-de-departamentos-en-ate-vitarte--lima--lima',
  'portal': 'urbania',
  'operacion': 'alquiler',
  'inmueble': 'departamentos'}]

Cuantos inmuebles segun el tipo de operacion hay en el distrito.

- por ejemplo: Total de casas en Miraflores: 600; Total de terrenos en Miraflores: 8 .. etc


In [6]:
num_dep = []

for web in web_distrito:
    
    enlace = web["web"]
    distrito = web["distrito"]
    inmueble = web["inmueble"]
    portal = web["portal"]
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")   # modo oculto
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
    driver.minimize_window()
    driver.get(enlace)
    
    elementos = driver.find_elements(By.CSS_SELECTOR, "h1.postingsTitle-module__title")
    
    if elementos:  # si encontró al menos uno
        texto = elementos[0].text  # Tomamos el primero
        total_inmuebles = int(''.join(filter(str.isdigit, texto)))
        # print(f"Total {inmueble} en {distrito}: {total_inmuebles}")
    else:
        total_inmuebles = 0 
        # print(f"Total {inmueble} en {distrito}: {total_inmuebles}")
    
    tqdm.write(f"Total {inmueble} en {distrito}: {total_inmuebles}")
    
    num_dep.append({
            "distrito": distrito,
            "numero_inmuebles": total_inmuebles,
            "inmueble": inmueble,
            "operacion" : operacion,
            "portal" : portal
            })
    
    driver.quit()


Total departamentos en Ancon: 0
Total departamentos en Ate Vitarte: 42


In [7]:
num = sum(i["numero_inmuebles"] for i in num_dep)
print(f"Total de {inmueble} en {operacion} son: {num}")

Total de departamentos en alquiler son: 42


In [8]:
paginas = []
for num_dep_i in num_dep:
    
    distrito = num_dep_i["distrito"]
    num_inmueble_i = num_dep_i["numero_inmuebles"]
    portal = num_dep_i["portal"]
    operacion = num_dep_i["operacion"]
    inmueble = num_dep_i["inmueble"]
    
    ## asumiendo que son 30 departamentos por pagina. 
    numero_pagina = (num_inmueble_i + 30 - 1) // 30
    
    
    if portal == "adondevivir":
        for pagina in range(1, numero_pagina+1):
            web = f"https://www.adondevivir.com/{inmueble}-en-{operacion}-en-{distrito_procesado(distrito)}-pagina-{pagina}.html" ## a donde vivir. 
            paginas.append({
            "distrito": distrito, 
            "web": web,
            "portal": portal,
            "operacion":operacion, 
            "inmueble": inmueble
            })

    else:
        for pagina in range(1, numero_pagina+1):
            web = f"https://urbania.pe/buscar/{operacion}-de-{inmueble}-en-{distrito_procesado(distrito)}--lima--lima?page={pagina}"
            paginas.append({
            "distrito": distrito, 
            "web": web,
            "portal": portal,
            "operacion":operacion, 
            "inmueble": inmueble
            })
    
## Resultado    
paginas[:3]

[{'distrito': 'Ate Vitarte',
  'web': 'https://urbania.pe/buscar/alquiler-de-departamentos-en-ate-vitarte--lima--lima?page=1',
  'portal': 'urbania',
  'operacion': 'alquiler',
  'inmueble': 'departamentos'},
 {'distrito': 'Ate Vitarte',
  'web': 'https://urbania.pe/buscar/alquiler-de-departamentos-en-ate-vitarte--lima--lima?page=2',
  'portal': 'urbania',
  'operacion': 'alquiler',
  'inmueble': 'departamentos'}]

## **Extracción de Datos**

In [9]:
for pagina in tqdm(paginas, desc="Procesando Distritos", unit="distrito"):
    
    portal = pagina["portal"]
    operacion = pagina["operacion"]
    inmueble = pagina["inmueble"]
    distrito = pagina["distrito"]
    
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")   # modo oculto
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    driver.get(pagina["web"])
    
    ## Capturando Informacion
    anuncios = driver.find_elements(By.CLASS_NAME, "postingsList-module__postings-container")
    
    
    for anuncio in anuncios:

        ## Identificacion de precio:
        precio = anuncio.find_elements(By.CLASS_NAME, "postingPrices-module__price-container")
        precio_ = []
        for p in precio:
            precio_.append(p.text)

Procesando Distritos: 100%|██████████| 2/2 [00:09<00:00,  4.78s/distrito]


In [10]:
data_final = []

for pagina in tqdm(paginas, desc="Procesando Distritos", unit="distrito"):
    
    portal = pagina["portal"]
    operacion = pagina["operacion"]
    inmueble = pagina["inmueble"]
    distrito = pagina["distrito"]
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")   # modo oculto
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    driver.get(pagina["web"])
    
    ## Capturando Informacion
    anuncios = driver.find_elements(By.CLASS_NAME, "postingsList-module__postings-container")
    
    data_vivienda = []
    for anuncio in anuncios:

        ## Identificacion de precio:
        precio = anuncio.find_elements(By.CLASS_NAME, "postingPrices-module__price-container")
        precio_ = []
        for p in precio:
            precio_.append(p.text)

        precio_f_ = []
        for p in precio_:
            monto = p.replace(",", "").strip()  # elimina comas y espacios
            partes = monto.split('·')  # divide soles y dólares
            partes_limpias = []
            for parte in partes:
                subpartes = parte.strip().split('\n')  # divide por salto de línea
                for sub in subpartes:
                    sub = sub.strip()
                    if sub:  # evitar vacíos
                        partes_limpias.append(sub)
            precio_f_.append(partes_limpias)

        soles = []
        dolares = []
        variacion = []

        for precios in precio_f_:  # precios = ['S/ 2200', 'USD 620', 'S/ 100 Mantenimiento']
            s = d = v = None
            for p in precios:
                if p.startswith('S/'):
                    if "Mantenimiento" not in p:
                        s = p.replace('S/', '').strip()
                elif p.startswith('USD'):
                    d = p.replace('USD', '').strip()
                elif "%" in p:
                    v = p.strip()
            soles.append(s)
            dolares.append(d)
            variacion.append(v)

        ## Mantenimiento 
        mantenimiento = anuncio.find_elements(By.CLASS_NAME, "postingPrices-module__expenses-property-listing")    
        mantenimiento_ = []
        for m in mantenimiento:
            mantenimiento_i = m.text
            mantenimiento_f = mantenimiento_i.replace("S/", "").replace("Mantenimiento", "").strip()
            mantenimiento_.append(mantenimiento_f)

        ## Caracteristica
        caracteristicas = anuncio.find_elements(By.CLASS_NAME, "postingMainFeatures-module__posting-main-features-block")
        caracteristicas_ = []
        for c in caracteristicas:
            caracteristicas_.append(c.text)

        area_ = []
        dorm_ = []
        banio_ = []
        est_ = []
        for c in caracteristicas_:
            carac = c.split("\n")
            area = dorm = banio = est = None
            for c_i in carac:
                if "m² tot." in c_i:
                    area = c_i.replace('m² tot.', '').strip()
                elif "dorm." in c_i:
                    dorm = c_i.replace('dorm.', '').strip()
                elif "baños" in c_i:
                    banio = c_i.replace('baños', '').strip()
                elif "estac." in c_i:
                    est = c_i.replace('estac.', '').strip()

            area_.append(area)
            dorm_.append(dorm)
            banio_.append(banio)
            est_.append(est)


        ## Detalle
        detalle = anuncio.find_elements(By.CLASS_NAME, "postingCard-module__posting-description")
        detalle_ = []
        for d in detalle:
            # print(d.text)
            detalle_.append(d.text)    

        ## Direccion
        direccion = anuncio.find_elements(By.CLASS_NAME, "postingLocations-module__location-block")
        direccion_ = []
        for d in direccion:
            direccion_.append(d.text)

        ## Link del anuncio
        descripcion = anuncio.find_elements(By.CLASS_NAME, "postingCard-module__posting-description")
        enlace_ = []
        for descripcion_i in descripcion:
            enlace = descripcion_i.find_element(By.TAG_NAME, "a").get_attribute("href")
            enlace_.append(enlace)


        data_vivienda_sub = []   
        for precio_i, soles_i, dolares_i, variacion_i, mantenimiento_i, direccion_i, caracteristica_i, area_i, dorm_i, banio_i, est_i, enlace_i, detalle_i in zip(precio_, soles, dolares, variacion, mantenimiento_, direccion_, caracteristicas_, area_, dorm_, banio_, est_, enlace_, detalle_):
            data_vivienda_sub.append({
                "fuente": portal,
                "inmueble": inmueble, 
                "operacion": operacion, 
                "distrito": distrito,
                "direccion": direccion_i,
                "precio": precio_i,
                "soles": soles_i,
                "dolares": dolares_i,
                "variacion": variacion_i,
                "mantenimiento": mantenimiento_i,
                "caracteristica": caracteristica_i,
                "area": area_i,
                "dormitorio": dorm_i,
                "baños": banio_i,
                "estacionamientos": est_i,
                "detalle": detalle_i,
                "enlace": enlace_i,
                "fecha": hoy 
            })


        data_vivienda.extend(data_vivienda_sub)
        
        driver.quit() 
    
    data_final.extend(data_vivienda)

Procesando Distritos: 100%|██████████| 2/2 [00:14<00:00,  7.05s/distrito]


In [12]:
num = sum(i["numero_inmuebles"] for i in num_dep)
print(f"Total de {inmueble} en {operacion} esperado: {num}")
print(f"Total de {inmueble} en {operacion} decargado: {len(data_final)}")
print(f"Porcentaje decargado: {round(len(data_final)/num*100, 2)} %")

Total de departamentos en alquiler esperado: 42
Total de departamentos en alquiler decargado: 42
Porcentaje decargado: 100.0 %


In [13]:
num = sum(i["numero_inmuebles"] for i in num_dep)
print(f"Total de {inmueble} en {operacion} esperado: {num}")
print(f"Total de {inmueble} en {operacion} decargado: {len(data_final)}")
print(f"Porcentaje decargado: {round(len(data_final)/num*100, 2)} %")

Total de departamentos en alquiler esperado: 42
Total de departamentos en alquiler decargado: 42
Porcentaje decargado: 100.0 %


In [14]:
data_final_df = pd.DataFrame(data_final)
pd.value_counts(data_final_df["distrito"])

C:\Users\PC\AppData\Local\Temp\ipykernel_30908\1226787397.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(data_final_df["distrito"])


distrito
Ate Vitarte    42
Name: count, dtype: int64

## **Nuevo: Extraccion de datos de cada departamento Registrado en el distrito i-esimo**

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [21]:
data_final[0]

{'fuente': 'urbania',
 'inmueble': 'departamentos',
 'operacion': 'alquiler',
 'distrito': 'Ate Vitarte',
 'direccion': 'Hernando de Soto 238\nSalamanca De Monterrico, Ate Vitarte',
 'precio': 'S/ 1,200',
 'soles': '1200',
 'dolares': None,
 'variacion': None,
 'mantenimiento': '',
 'caracteristica': '65 m² tot.\n1 dorm.\n1 baño',
 'area': '65',
 'dormitorio': '1',
 'baños': None,
 'estacionamientos': None,
 'detalle': 'Lindo departamento en alquiler con una excelente ubicación, cerca a Plaza Vea de Circunvalación, cerca a parque y a una cuadra de la Av. Los Quechuas. Cuenta con una amplia sala comedor, iluminada, cocina estilo americana con reposteros altos y bajos y con acabados de granito, también bastante iluminada, 1 habitación amplia, baño del departamento; todos los espacios son bien distribuidos, y bastantes iluminados como ventilados, además de ello cuenta con cómodo patio y su lavandería.',
 'enlace': 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-

In [36]:
enlace = []

for i in data_final:
    enlace.append(i["enlace"])
    
enlace[:5]

['https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-salamanca-de-monterrico-ate-vitarte-1-dormitorio-65019992?n_src=Listado&n_pills=Seguridad&n_pg=1&n_pos=1',
 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-ate-vitarte-lima-3-dormitorios-148160219?n_src=Listado&n_pills=Guardian%C3%ADa%2FSeguridad+privada&n_pg=1&n_pos=2',
 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-salamanca-de-monterrico-ate-vitarte-2-dormitorios-148069607?n_src=Listado&n_pills=Cerca+a+colegios&n_pg=1&n_pos=3',
 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-javier-prado-et6-ate-vitarte-3-dormitorios-145317529?n_src=Listado&n_pills=Ascensor&n_pg=1&n_pos=4',
 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-mayorazgo-etapa-iii-ate-vitarte-2-dormitorios-147989028?n_src=Listado&n_pills=Guardian%C3%ADa%2FSeguridad+privada&n_pg=1&n_pos=5']

In [37]:
enlace_anuncio = [{"enlace_casa": vivienda_i["enlace"]
                , "direccion": vivienda_i["direccion"]
                , "fuente": vivienda_i["fuente"]
                , "inmueble": vivienda_i["inmueble"]} for vivienda_i in data_final]

enlace_anuncio[0]["enlace_casa"]

'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-salamanca-de-monterrico-ate-vitarte-1-dormitorio-65019992?n_src=Listado&n_pills=Seguridad&n_pg=1&n_pos=1'

In [ ]:
for enlace_i in enlace_anuncio[:1]:
    url = enlace_i["enlace_casa"]
    driver.get(url)

Descripcion

In [46]:
enlace = 'https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-salamanca-de-monterrico-ate-vitarte-1-dormitorio-65019992?n_src=Listado&n_pills=Seguridad&n_pg=1&n_pos=1'

In [47]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")   # modo oculto
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options = options)
# driver.minimize_window()
driver.get(enlace)

### Descripcion del Elemento

In [50]:
# Buscar la descripción por la clase
descripcion_element = driver.find_element(By.CLASS_NAME, "description-module__wrapper-description___2rEoY")
descripcion_texto = descripcion_element.text

print("Descripción extraída:\n", descripcion_texto)

Descripción extraída:
 Alquiler de Departamento en 1er Piso, Salamanca
Lindo departamento en alquiler con una excelente ubicación, cerca a Plaza Vea de Circunvalación, cerca a parque y a una cuadra de la Av. Los Quechuas.
Cuenta con una amplia sala comedor, iluminada, cocina estilo americana con reposteros altos y bajos y con acabados de GRANITO, también bastante iluminada, 1 habitación amplia, baño del departamento; todos los espacios son bien distribuidos, y bastantes iluminados como ventilados, además de ello cuenta con cómodo patio y su lavandería.
**


### Caracteristicas Generales del Inmueble

In [51]:
caracteristicas_element = driver.find_element(By.CLASS_NAME, "generalFeaturesProperty-module__description-container___15yV-")
caracteristicas_texto = caracteristicas_element.text
print("Características extraídas:\n", caracteristicas_texto)

Características extraídas:
 Caseta de guardia
Terraza
Seguridad
Centros Comerciales Cercanos
Cerca a colegios
Mascotas
Reposteros en cocina
Piso en el que se encuentra : 1
Intercomunicador
Av. acceso asfaltada
Av. acceso afirmada
Cerca a Parque (a menos de 2 cdras)
Cocina


### Direccion

In [55]:
informacion_anunciante_element = driver.find_element(By.CLASS_NAME, "section-location-property")
anunciante_texto = informacion_anunciante_element.text
print("Características extraídas:\n", anunciante_texto)

Características extraídas:
 Hernando de Soto 238, Salamanca De Monterrico, Ate Vitarte


### Imagen Mapa

In [58]:
map_element = driver.find_element(By.ID, 'static-map')
image_url = map_element.get_attribute('src')
image_url

'https://img10.naventcdn.com/ficha/map/Urbania/65019992E.png'

### Imagenes

## **Guardar Datos**

In [ ]:
ruta_salida = rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\6.Analisis_Alquiler_Venta\data\data_{operacion}_{inmueble}_{portal}.csv"

Tipo json

In [ ]:
import json

with open(f"data_{operacion}_{inmueble}_{portal}.json", "w", encoding="utf-8") as f:
    json.dump(data_final, f, indent=4, ensure_ascii=False, default=str)

Tipo CSV

In [ ]:
data_final_df.to_csv(ruta_salida, sep = "|",index=False)